# GIST Driver
##Test on events of interest:
    texnet2021zjsk with a duplicate with the date changed to whatever today is
Assumes InjectionSQLScheduled completed

#Configuration

##Libraries

In [0]:
! pip install geopandas
! pip install geodatasets
! pip install contextily
#! pip install folium matplotlib mapclassify contextily

  Using cached geopandas-0.14.3-py3-none-any.whl (1.1 MB)
  Using cached fiona-1.9.6-cp39-cp39-manylinux2014_x86_64.whl (15.7 MB)
  Using cached shapely-2.0.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.5 MB)
  Using cached pyproj-3.6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (8.3 MB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-7316a044-4353-4309-9d4b-32c8cf66240f/bin/python -m pip install --upgrade pip' command.
  Using cached geodatasets-2023.12.0-py3-none-any.whl (19 kB)
  Using cached pooch-1.8.1-py3-none-any.whl (62 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-7316a044-4353-4309-9d4b-32c8cf66240f/bin/python -m pip install --upgrade pip' command.
  Using cached contextily-1.6.0-py3-none-any.whl (17 kB)
  Using cached xyzservices-2024.4.0-py3-none-any.whl (

In [0]:
import sys
basePath='/Workspace/Users/bill.curry@exxonmobil.com'
gistPath='/Workspace/Users/bill.curry@exxonmobil.com/gist'
sys.path.append(basePath)
import dataBricksConfig as db

In [0]:
import numpy as np
import pandas as pd
import os
import gistMCLive as gi
import eqSQL as es
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
#import contextily as cx

In [0]:
import gistPlots as gp

## Parameters

### Input Files
Produced from injectionSQLScheduled.py

In [0]:
injPath=gistPath+'/ScheduledOutput'

DeepWellFile=injPath+'/deep.csv'
DeepInjFile=injPath+'/deepReg.csv'
ShallowWellFile=injPath+'/shallow.csv'
ShallowInjFile=injPath+'/shallowReg.csv'
runPath=gistPath+'/Outputs/'

################################################
# Output prefix for realizations of parameters #
################################################
RealizationPrefix=runPath+'MC'

###General

In [0]:
######################
# General Parameters #
######################
nRealizations=20
nTimeBins=21
WaterDensity=1020.
WaterViscosity=0.001
VerticalCompressibility=0.000000001
FluidCompressibility=0.00000000036
FaultFrictionCoeff=0.6
RockFrictionCoeff=0.6

###################################
# General Parameter Uncertainties #
###################################
WaterDensityUncertainty=5.
WaterViscosityUncertainty=0.000001
VerticalCompressibilityUncertainty=0.00000000001
FluidCompressibilityUncertainty=0.000000000001
FaultFrictionCoeffUncertainty=0.05
RockFrictionCoeffUncertainty=0.05

###Shallow interval

In [0]:
###############################
# Shallow Interval Parameters #
###############################
ShallowPorosityPercent=15.
ShallowPermMD=10.
ShallowThicknessFT=3700.
# From Lei's excel sheet for Stanton - Young's modulus 5.83 GPA
# Shear = Young / 2(1+ Poisson's ratio)
ShallowShearModulus=2.4e9
# Is Dynamic undrained?
ShallowPoissonsRatioDrained=0.19
ShallowPoissonsRatioUndrained=0.215
# No idea - probably needs updating
ShallowBiotsCoefficient=0.31

##################################
# Shallow Interval Uncertainties #
##################################
ShallowPorosityPercentUncertainty=8.
# I think this should be a flat distribution bounds set using the logarithm of the permeability - now implemented
ShallowPermLogMDUncertainty=1.
ShallowThicknessFTUncertainty=500.
ShallowShearModulusUncertainty=1e9
ShallowPoissonsRatioDrainedUncertainty=0.01
ShallowPoissonsRatioUndrainedUncertainty=0.01
ShallowBiotsCoefficientUncertainty=0.05

### Deep Interval

In [0]:
###############################
# Deep Interval Parameters #
###############################
DeepPorosityPercent=8.
# Pulse pressure test shows background diffusivity of 0.5 m2/s
# Back that into permeaiblity: k = D * eta * (Cr + porosity Cp)
# 0.5 x 0.001 x (1e -9 + 0.08 x 3.6 e-10) = 5.144 e-13
# / 9.9e-16 m2 to mD = 520mD!
DeepPermMD=100.
# Now let's change this to something where 1D is the high range and 10mD is the low range

DeepThicknessFT=1400.
# Roughly double of shallow - 
DeepShearModulus=5e9
DeepPoissonsRatioDrained=0.30
DeepPoissonsRatioUndrained=0.31
DeepBiotsCoefficient=0.31

##################################
# DeepInterval Uncertainties #
##################################
DeepPorosityPercentUncertainty=5.
DeepPermLogMDUncertainty=1.
DeepThicknessFTUncertainty=200.
DeepShearModulusUncertainty=1e9
DeepPoissonsRatioDrainedUncertainty=0.005
DeepPoissonsRatioUndrainedUncertainty=0.005
DeepBiotsCoefficientUncertainty=0.05

#Earthquake Selection

In [0]:
eqs=es.eqSQL()
EQDF1=eqs.getEarthquake('texnet2021zjsk')
EQDF1['Strike']=96
EQDF1['Dip']=81.
EQDF1['Rake']=-24.
EQDF2=EQDF1.copy(deep=True)
EQDF2['Origin Date']=pd.to_datetime(pd.Timestamp('today')).date()
EQDF2.at[0,'EventId'] = 'texnet2024FAKE'
EQDF=pd.concat([EQDF1,EQDF2],ignore_index=True)
EQDF=EQDF.rename(columns={'LatitudeErrorKm':'LatitudeError','LongitudeErrorKm':'LongitudeError','EventId':'EventID'})
# We could make use of the Rake
EQDF.info()
EQDF.to_csv(runPath+'/EQToday.csv')
print(EQDF)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 35 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   SeismicEventId              2 non-null      int64         
 1   DataSource                  2 non-null      object        
 2   DataSourceUrl               0 non-null      object        
 3   EventID                     2 non-null      object        
 4   EventTimeUtc                2 non-null      datetime64[ns]
 5   EventTimeInLocalTimeZone    2 non-null      object        
 6   EventTimeZone               2 non-null      object        
 7   EventType                   2 non-null      object        
 8   DepthKm                     2 non-null      float64       
 9   DepthErrorKm                2 non-null      float64       
 10  Magnitude                   2 non-null      float64       
 11  MagnitudeError              0 non-null      float64   

#GIST Initialization

## Shallow 

In [0]:
########################################
# Initialize GIST for shallow interval #
########################################
shallowM=gi.gistMC(nReal=nRealizations,
                   ntBin=nTimeBins)

In [0]:
##############################################################
# Initialize pore pressure model                             #
# Parameters mostly from Katie Smye, are rough at this point #
##############################################################
shallowM.initPP(rho_0=WaterDensity,
                nta=WaterViscosity,
                phi=ShallowPorosityPercent,
                kMD=ShallowPermMD,
                h=ShallowThicknessFT,
                alphav=VerticalCompressibility,
                beta=FluidCompressibility)
######################
# Uncertainty ranges #
######################
shallowM.initMCPP(rhoUnc=WaterDensityUncertainty,
                  ntaUnc=WaterViscosityUncertainty,
                  phiUnc=ShallowPorosityPercentUncertainty,
                  kMDLogUnc=ShallowPermLogMDUncertainty,
                  hUnc=ShallowThicknessFTUncertainty,
                  alphavUnc=VerticalCompressibilityUncertainty,
                  betaUnc=FluidCompressibilityUncertainty)

  logSpace: lower bound:  1.0 , upper bound:  100.0


In [0]:
######################################################
# These parameters need more vetting for Tarzan area #
######################################################
shallowM.initPE(mu=ShallowShearModulus,
                nu=ShallowPoissonsRatioDrained,
                nu_u=ShallowPoissonsRatioUndrained,
                alpha=ShallowBiotsCoefficient,
                mu_f=FaultFrictionCoeff,
                mu_r=RockFrictionCoeff,
                match=True)
######################
# Uncertainty ranges #
######################
shallowM.initMCPE(muUnc=ShallowShearModulusUncertainty,
                  nuUnc=ShallowPoissonsRatioDrainedUncertainty,
                  nu_uUnc=ShallowPoissonsRatioUndrainedUncertainty,
                  alphaUnc=ShallowBiotsCoefficientUncertainty,
                  muFUnc=FaultFrictionCoeffUncertainty,
                  muRUnc=RockFrictionCoeffUncertainty,
                  match=True)

## Deep

In [0]:
##################################################
# Initialize 2nd GIST instance for deep interval #
##################################################
deepM=gi.gistMC(nReal=nRealizations,
                ntBin=nTimeBins)
##################################################################
# Initialize pore pressure model                                 #
##################################################################
deepM.initPP(rho_0=WaterDensity,
             phi=DeepPorosityPercent,
             kMD=DeepPermMD,
             h=DeepThicknessFT,
             alphav=VerticalCompressibility,
             beta=FluidCompressibility)

deepM.initMCPP(rhoUnc=WaterDensityUncertainty,
               ntaUnc=WaterViscosityUncertainty,
               phiUnc=DeepPorosityPercentUncertainty,
               kMDLogUnc=DeepPermLogMDUncertainty,
               hUnc=DeepThicknessFTUncertainty,
               alphavUnc=VerticalCompressibilityUncertainty,
               betaUnc=FluidCompressibilityUncertainty)

  logSpace: lower bound:  10.0 , upper bound:  1000.0


In [0]:
######################################################
# These parameters need more vetting for Tarzan area #
######################################################
deepM.initPE(mu=DeepShearModulus,
             nu=DeepPoissonsRatioDrained,
             nu_u=DeepPoissonsRatioUndrained,
             alpha=DeepBiotsCoefficient,
             mu_f=FaultFrictionCoeff,
             mu_r=RockFrictionCoeff,
             match=True)
deepM.initMCPE(muUnc=DeepShearModulusUncertainty,
               nuUnc=DeepPoissonsRatioDrainedUncertainty,
               nu_uUnc=DeepPoissonsRatioUndrainedUncertainty,
               alphaUnc=DeepBiotsCoefficientUncertainty,
               muFUnc=FaultFrictionCoeffUncertainty,
               muRUnc=RockFrictionCoeffUncertainty,
               match=True)

In [0]:
#######################################
# Add uncertainties to both intervals #
#######################################

######################
# Post diffusivities #
######################
print("Shallow Diffusivity="+str(shallowM.diffPPMax))
print("Deep Diffusivity="+str(deepM.diffPPMax))


Shallow Diffusivity=0.08995573645408184
Deep Diffusivity=0.9172847108545967


#Well selection

####Input wells

In [0]:
#########################################
# Add potential wells to each instance  #
#########################################
print("Before adding shallow wells")
#shallowM.addWells(ShallowWellFile,ShallowInjFile,verbose=1)

Before adding shallow wells


In [0]:
print("Before adding deep wells")
deepM.addWells(DeepWellFile,DeepInjFile,verbose=1)

Before adding deep wells
 gistMC.addWells: well file added with  2136  wells
 gistMC.addWells: well columns: Index(['ID', 'InjectionWellId', 'APINumber', 'UICNumber', 'Basin', 'County',
       'SurfaceHoleLatitude', 'SurfaceHoleLongitude', 'WellName', 'WellNumber',
       'InjectionType', 'CompletedWellDepthClassification', 'InjectionStatus',
       'CompletionAndDrillingPermitStatus', 'StartDate', 'PermitIsAmendment',
       'DaysToPermitApproval', 'PermittedMaxLiquidBPD',
       'PermittedMaxLiquidPSIG', 'PermittedIntervalBottomFt',
       'PermittedIntervalTopFt', 'PermittedCommercialStatus',
       'PermittedWellDepthClassification', 'Operator', 'OperatorType',
       'OperatorPrincipalCompany', 'OperatorPrincipalCompanyType',
       'PermitStage', 'SRAOrSIR', 'UploadTime'],
      dtype='object')
 gistMC.addWells: injection file has  1791  unique wells
 gistMC.addWells: injection file first day:  3880.0
 gistMC.addWells: injection file last day:  19820.0
 gistMC.addWells: injection

In [0]:
#shallowM.wellDF.info()
#EQDF.info()

In [0]:
#eq=EQDF.iloc[1]
#print(eq)
#shallowPPSelectedWells,shallowPPIgnoredWells,shallowPPInj=shallowM.findWells(eq,PE=False)
#shallowPESelectedWells,shallowPEIgnoredWells,shallowPEInj=shallowM.findWells(eq,PE=True)
#shallowPPSelectedWells['Selected']=True
#shallowPPIgnoredWells['Selected']=False
#shallowPPWells=pd.concat([shallowPPSelectedWells,shallowPPIgnoredWells])
#shallowPPWells['EventID']=eq['EventID']
#shallowPESelectedWells['Selected']=True
#shallowPEIgnoredWells['Selected']=False
#shallowPEWells=pd.concat([shallowPESelectedWells,shallowPEIgnoredWells])
#shallowPEWells['EventID']=eq['EventID']

In [0]:
eq=EQDF.iloc[0]
print(eq)

deepPPSelectedWells,deepPPIgnoredWells,deepPPInj=deepM.findWells(eq,PE=False)
#deepPESelectedWells,deepPEIgnoredWells,deepPEInj=deepM.findWells(eq,PE=True)
deepPPSelectedWells['Selected']=True
deepPPIgnoredWells['Selected']=False
deepPPWells=pd.concat([deepPPSelectedWells,deepPPIgnoredWells])
deepPPWells['EventID']=eq['EventID']
#deepPESelectedWells['Selected']=True
#deepPEIgnoredWells['Selected']=False
#deepPEWells=pd.concat([deepPESelectedWells,deepPEIgnoredWells])
#deepPEWells['EventID']=eq['EventID']

SeismicEventId                                       119341
DataSource                        TexNet Earthquake Catalog
DataSourceUrl                                          None
EventID                                      texnet2021zjsk
EventTimeUtc                            2021-12-28 01:55:43
EventTimeInLocalTimeZone               2021-12-27T19:55:43Z
EventTimeZone                                           CST
EventType                                        Earthquake
DepthKm                                             8.43689
DepthErrorKm                                       0.578692
Magnitude                                           4.56297
MagnitudeError                                          NaN
MagnitudeType                                    ml(texnet)
Location                                      Western Texas
Status                                                final
UpdatedDateUtc                          2022-01-03 15:28:00
StationCount                            

/Workspace/Users/bill.curry@exxonmobil.com/gist/gistMCLive.py:727: RuntimeWarning: invalid value encountered in sqrt
  diffusionDistances[iw]=0.001*np.sqrt(4.*np.pi*self.diffPPMax*injectionDays*24*60*60)


In [0]:
#shallowPPWells.to_csv(runPath+'/StantonToday_ShallowPPWells.csv')
#shallowPEWells.to_csv(runPath+'/StantonToday_ShallowPEWells.csv')
#deepPPWells.to_csv(runPath+'/StantonToday_DeepPPWells.csv')
#deepPEWells.to_csv(runPath+'/StantonToday_DeepPEWells.csv')

In [0]:
# Dump injection data to file:
#deepPPInj.to_csv(runPath+'/StantonToday_DeepPPInj.zip')
#shallowPPInj.to_csv(runPath+'/StantonToday_ShallowPPInj.zip')

In [0]:
#eq.loc['Origin Date']=pd.to_datetime(eq['Origin Date'])

In [0]:
#import gistPlots as gp

#gp.wellInjPlot(deepPPInj,deepPPWells,'01-01-2018','02-11-2023','Deep Injection',True)
#gp.wellInjPlot(shallowPPInj,shallowPPWells,'01-01-2018','02-11-2023','Shallow Injection',True)

#fig, ax = plt.subplots(figsize=(12,24))
# Merge well information onto injection rates
# Need to incorporate time sampling other than 10 day


#Initial GIST runs


In [0]:

import time
time1 = time.time()
dWellPPScenarios2021b=deepM.runPressureScenarios(EQDF.iloc[0],deepPPSelectedWells,deepPPInj,verbose=1)
time2 = time.time()

 gistMC.runPressureScenarios: Number of wells considered:  485
runPressureScenarios: 
 gistMC.pressureScenario: well  0  of  485 :  1007812
 gistMC.pressureScenario: well  10  of  485 :  1003161
 gistMC.pressureScenario: well  20  of  485 :  1004738
 gistMC.pressureScenario: well  30  of  485 :  2121097
 gistMC.pressureScenario: well  40  of  485 :  2121094
 gistMC.pressureScenario: well  50  of  485 :  2112175
 gistMC.pressureScenario: well  60  of  485 :  2054500
 gistMC.pressureScenario: well  70  of  485 :  2042738
 gistMC.pressureScenario: well  80  of  485 :  2111358
 gistMC.pressureScenario: well  90  of  485 :  2093666
 gistMC.pressureScenario: well  100  of  485 :  2118060
 gistMC.pressureScenario: well  110  of  485 :  2111864
 gistMC.pressureScenario: well  120  of  485 :  2043314
 gistMC.pressureScenario: well  130  of  485 :  2022280
 gistMC.pressureScenario: well  140  of  485 :  2120517
 gistMC.pressureScenario: well  150  of  485 :  2064744
 gistMC.pressureScenario: wel

In [0]:
time3 = time.time()
dWellPPScenarios2021fast,dPPTimeSeriesT2021fast=deepM.runPressureScenariosFast(EQDF.iloc[0],deepPPSelectedWells,deepPPInj,verbose=1)
time4 = time.time()
print(time2-time1,' for original')
print(time4-time3,' for fast')

/Workspace/Users/bill.curry@exxonmobil.com/gist/gistMCLive.py:1212: RuntimeWarning: divide by zero encountered in true_divide
  epp=sc.exp1(ppp.reshape((nwC,self.nReal,1)).repeat(nt0,2) / durations[:].reshape((1,1,nt0)).repeat(nwC,0).repeat(self.nReal,1))


pressureScenariosToDF:  485 20 9700 20
pressureScenariosToDF:  9700 9700
25.321803092956543  for original
122.7026994228363  for fast


In [0]:
# Run deep scenarios
#eq['Origin Date']=pd.to_datetime(eq['Origin Date'])
#dWellPPScenariosToday,dPPTimeSeriesToday=deepM.runPressureScenarios(EQDF.iloc[1],deepPPSelectedWells,deepPPInj,verbose=1)

time5 = time.time()
dWellPPScenarios2021slow,dPPTimeSeriesT2021slow=deepM.runPressureScenariosSlow(EQDF.iloc[0],deepPPSelectedWells,deepPPInj,verbose=1)
time6 = time.time()
print(time6-time5,' for slow')


 runPressureScenariosFast - time step  1  of  1595
 runPressureScenariosFast - time step  2  of  1595
 runPressureScenariosFast - time step  3  of  1595
 runPressureScenariosFast - time step  4  of  1595
 runPressureScenariosFast - time step  5  of  1595
 runPressureScenariosFast - time step  6  of  1595
 runPressureScenariosFast - time step  7  of  1595
 runPressureScenariosFast - time step  8  of  1595
 runPressureScenariosFast - time step  9  of  1595
 runPressureScenariosFast - time step  10  of  1595
 runPressureScenariosFast - time step  11  of  1595
 runPressureScenariosFast - time step  12  of  1595
 runPressureScenariosFast - time step  13  of  1595
 runPressureScenariosFast - time step  14  of  1595
 runPressureScenariosFast - time step  15  of  1595
 runPressureScenariosFast - time step  16  of  1595
 runPressureScenariosFast - time step  17  of  1595
 runPressureScenariosFast - time step  18  of  1595
 runPressureScenariosFast - time step  19  of  1595
 runPressureScenarios

In [0]:

baseOutPath='/Workspace/Users/bill.curry@exxonmobil.com/gist/Outputs/StantonToday_'
#dWellPPScenariosToday.to_csv(baseOutPath+'deepPP.csv')
dWellPPScenarios2021b.to_csv(baseOutPath+'2021deepPPoriginal.csv')
dWellPPScenarios2021fast.to_csv(baseOutPath+'2021deepPPfast.csv')
dWellPPScenarios2021slow.to_csv(baseOutPath+'2021deepPPslow.csv')
np.save(baseOutPath+'deepPPTimeSeries2021fast.npy',dPPTimeSeriesT2021fast)
np.save(baseOutPath+'deepPPTimeSeries2021slow.npy',dPPTimeSeriesT2021slow)
#sWellPPScenariosToday.to_csv(baseOutPath+'shallowPP.csv')
#sWellPPScenarios2021.to_csv(baseOutPath+'2021shallowPP.csv')

In [0]:
sWellPPScenariosToday,sPPTimeSeriesToday=shallowM.runPressureScenarios(EQDF.iloc[1],shallowPPSelectedWells,shallowPPInj,verbose=1)
sWellPPScenarios2021,sPPTimeSeries2021=shallowM.runPressureScenarios(EQDF.iloc[0],shallowPPSelectedWells,shallowPPInj,verbose=1)

In [0]:
dPPTimeSeriesSum=dPPTimeSeriesT2021a.sum(axis=1)
#sPPTimeSeriesSum=sPPTimeSeriesToday.sum(axis=1)
dPPTimes=np.linspace(deepM.injOT, deepM.injOT+deepM.injDT*(deepM.injNT-1.),deepM.injNT)
#sPPTimes=np.linspace(shallowM.injOT, shallowM.injOT+shallowM.injDT*(shallowM.injNT-1.),shallowM.injNT)

In [0]:
#np.save(baseOutPath+'deepPPTimeSeriesToday.npy',dPPTimeSeriesT2021a)
#np.save(baseOutPath+'deepPPTimeSeriesToday.npy',dPPTimeSeriesToday)
#np.save(baseOutPath+'shallowPPTimes.npy',sPPTimes)
#np.save(baseOutPath+'shallowPPTimeSeriesSum.npy',sPPTimeSeriesSum)
np.save(baseOutPath+'deepPPTimeSeriesSum.npy',dPPTimeSeriesSum)

In [0]:
# Window out significant wells - save individual time series for those


In [0]:
deepWindowedWells=dWellPPScenariosToday[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
shallowWindowedWells=sWellPPScenariosToday[sWellPPScenarios['Pressures']>0.1]['ID'].unique()

In [0]:
# dPPTimeSeries is time, well, realization - need top wells with time series of pressures
dPPWellIndices=deepPPWellsToday['ID'].isin(deepWindowedWells)
print(sum(dPPWellIndices),' wells out of ',len(dPPWellIndices),' have >0.1 PSI contribution')

In [0]:

dPPTimeSeriesSelected=dPPTimeSeriesToday[:,dPPWellIndices,:]
dPPSelectedWellNameDF=deepPPWellsToday['WellName'][dPPWellIndices]
print(dPPSelectedWellNameDF.to_list())

In [0]:
i=0
for wellName in dPPSelectedWellNameDF.to_list():
  np.save(baseOutPath+'deepPPTimeSeries'+wellName+'.npy',dPPTimeSeriesSelected[:,i,:])
  i=i+1
  print('Well '+wellName+' written '+i)
dPPSelectedWellNameDF.to_csv(baseOutPath+'deepSelectedWellNames.csv')

In [0]:
# sPPTimeSeries is time, well, realization - need top wells with time series of pressures
sPPWellIndices=shallowPPWells['ID'].isin(shallowWindowedWellsToday)
print(sum(sPPWellIndices),' wells out of ',len(sPPWellIndices),' have >0.1 PSI contribution')
sPPTimeSeriesSelected=sPPTimeSeriesToday[:,sPPWellIndices,:]
sPPSelectedWellNameDF=shallowPPWellsToday['WellName'][sPPWellIndices]
print(sPPSelectedWellNameDF.to_list())


In [0]:
i=0
for wellName in sPPSelectedWellNameDF.to_list():
  np.save(baseOutPath+'shallowPPTimeSeries'+wellName+'.npy',sPPTimeSeriesSelected[:,i,:])
  i=i+1
  print('Well '+wellName+' written '+str(i))
sPPSelectedWellNameDF.to_csv(baseOutPath+'shallowSelectedWellNames.csv')

In [0]:
dWellPEScenariosToday=deepM.runPoroelasticScenarios(EQDF.iloc[1],deepPESelectedWells,deepPEInj,verbose=1)
dWellPEScenarios2021=deepM.runPoroelasticScenarios(EQDF.iloc[0],deepPESelectedWells,deepPEInj,verbose=1)

In [0]:
sWellPEScenariosToday=shallowM.runPoroelasticScenarios(EQDF.iloc[1],shallowPESelectedWells,shallowPEInj,verbose=1)
sWellPEScenarios2021=shallowM.runPoroelasticScenarios(EQDF.iloc[0],shallowPESelectedWells,shallowPEInj,verbose=1)

In [0]:

dWellPEScenariosToday.to_csv(baseOutPath+'deepPE.csv')
dWellPEScenarios2021.to_csv(baseOutPath+'2021deepPE.csv')

In [0]:

sWellPEScenariosToday[sWellPEScenariosToday['Realization']<250].to_csv(baseOutPath+'shallowPE0.csv')
sWellPEScenariosToday[sWellPEScenariosToday['Realization']>250].to_csv(baseOutPath+'shallowPE1.csv')
sWellPEScenarios2021[sWellPEScenarios2021['Realization']<250].to_csv(baseOutPath+'2021shallowPE0.csv')
sWellPEScenarios2021[sWellPEScenarios2021['Realization']>250].to_csv(baseOutPath+'2021shallowPE1.csv')


In [0]:
sWellPEScenariosToday.info()

In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
maxPressureList=[]
for ID in deepWindowedWells:
  maxPressure=max(filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID]['Pressures'])
  nReal=filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID].shape[0]
  print(ID,maxPressure,nReal)
  maxPressureList.extend([maxPressure,]*nReal)
filtDWellPPScenarios['MaxPressure']=maxPressureList
filtDWellPPScenarios=filtDWellPPScenarios.sort_values(by='MaxPressure',ascending=False)
shallowWindowedWells=sWellPPScenarios[sWellPPScenarios['Pressures']>0.1]['ID'].unique()
filtSWellPPScenarios=sWellPPScenarios[sWellPPScenarios['ID'].isin(shallowWindowedWells)]
maxPressureList=[]
for ID in shallowWindowedWells:
  maxPressure=max(filtSWellPPScenarios[filtSWellPPScenarios['ID']==ID]['Pressures'])
  nReal=filtSWellPPScenarios[filtSWellPPScenarios['ID']==ID].shape[0]
  print(ID,maxPressure,nReal)
  maxPressureList.extend([maxPressure,]*nReal)
filtSWellPPScenarios['MaxPressure']=maxPressureList
filtSWellPPScenarios=filtSWellPPScenarios.sort_values(by='MaxPressure',ascending=False)

In [0]:
filtDWellPPScenarios=filtDWellPPScenarios.merge(deepPPWells,on='ID')
filtSWellPPScenarios=filtSWellPPScenarios.merge(shallowPPWells,on='ID')
print('deep ',filtDWellPPScenarios.shape[0],', shallow',filtSWellPPScenarios.shape[0])

## Save Results

In [0]:
# Save results - visualize separately!
baseOutPath='/Workspace/Users/bill.curry@exxonmobil.com/gist/Outputs/StantonToday_'
#dWellPPScenarios.to_csv(baseOutPath+'deepPP.csv')
dWellPEScenarios.to_csv(baseOutPath+'deepPE.csv')
#sWellPPScenarios.to_csv(baseOutPath+'shallowPP.csv')
sWellPEScenarios.to_csv(baseOutPath+'shallowPE.csv')
#np.save(baseOutPath+'deepPPTimeSeries0.csv',dPPTimeSeries['Realization'<100])
#np.save(baseOutPath+'shallowPPTimeSeries.csv',sPPTimeSeries)

## Sanity check of initial results
### Check injection on wells with non-negligable pressure contributions

In [0]:
# Select injection histories of wells
deepWindowedInjection=deepPPInj[deepPPInj['ID'].is_in(deepWindowedWells)]
shallowWindowedInjection=shallowPPInj[shallowPPInj['ID'].is_in(shallowWindowedWells)]
deepInjQC=deepWindowedInjection.merge(deepPPWells[['Name','Operator','APINumber','Latitude','Longitude']],on='ID',how='left')
shallowInjQC=shallowWindowedInjection.merge(shallowPPWells[['Name','Operator','APINumber','Latitude','Longitude']],on='ID',how='left')

In [0]:
# Make stack plots of recent injection - Deep
injQC=deepInjQC
injQC['Date']=pd.to_datetime(injQC.Date)
# Generate x and y
dates=injQC['Date'].drop_duplicates()
# Loop over wells
oDate=min(dates)
eDate=max(dates)
dateRange=pd.date_range(start=oDate,end=eDate, freq='10D')
# Get x min, x max
wellIDs=injQC['ID'].drop_duplicates()
allWellBPDs=[]
allWellLabels=[]
for well in wellIDs:
  wellDates=injQC[injQC['ID']==well]['Date']
  wellBPDs=injQC[injQC['ID']==well]['BPD']
  minWD=min(wellDates)
  maxWD=max(wellDates)
  nPrepend=sum(dateRange<minWD)
  nAppend=sum(dateRange>maxWD)
  #print('Well ',well,' ',minWD,maxWD,oDate,eDate,nPrepend,' prePad, ',nAppend,' postPad, ',len(dateRange))
  #print('allWellBPDs: ',allWellBPDs)
  if nPrepend>0:
    prependZeroBPDs=pd.Series(np.zeros([nPrepend]))
  else:
    prependZeroBPDs=None
  if nAppend>0:
    appendZeroBPDs=pd.Series(np.zeros([nAppend,1]))
  else:
    appendZeroBPDs=None
  padWellBPDs=pd.concat([prependZeroBPDs,wellBPDs,appendZeroBPDs])
  allWellBPDs.append(padWellBPDs)
  #allWellLabels.append(injQC['WellName'][injQC['ID']==well][0])
  allWellLabels.append(injQC['WellName'][injQC['ID']==well].loc[0])
# Now have allWellBPDs, allWellLabels, and dateRange all set
# Generate Stacked area plot of injection
# y needs to be defined for each well, and extended for the full time series
col = sns.color_palette("hls", len(wellIDs))
#zoomDate=oneEQ['Origin Date'] - pd.DateOffset(years=2)
ax.stackplot(dateRange,allWellBPDs, labels=allWellLabels,colors=col)
ax.set_title('Selected well injection')
ax.set_xlabel('Date')
ax.set_ylabel('Rate (BPD)')
#if startDate!=None or endDate!=None: axes.xlim(startDate,endDate)
ax.legend(loc='upper left')
plt.show()

In [0]:
# Repeat for Shallow

In [0]:
# Plot Time Series of total pressures and injection histories


In [0]:

print(deepWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:

print(shallowWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:
# Concatenate pore pressure and poroelastic results
dWellPPScenarios['Physics']='Pore Pressure'
dWellPEScenarios['Physics']='Poroelastic'
dWellPPScenarios['Stresses']=dWellPPScenarios['Pressures']
allDeep=pd.concat(objs=[dWellPPScenarios,dWellPEScenarios[dWellPEScenarios['ID'].isin(dWellPPScenarios['ID'])]])
allDeep=allDeep.merge(deepPEWells,on='ID')



In [0]:
deepWindowedWells=dWellPPScenarios[dWellPPScenarios['Pressures']>0.1]['ID'].unique()
print(deepWindowedWells)
filtDWellPPScenarios=dWellPPScenarios[dWellPPScenarios['ID'].isin(deepWindowedWells)]
filtDWellPEScenarios=dWellPEScenarios[dWellPEScenarios['ID'].isin(deepWindowedWells)]
print(len(dWellPPScenarios),len(filtDWellPPScenarios),len(filtDWellPEScenarios))

In [0]:
maxPressureList=[]
for ID in deepWindowedWells:
  maxPressure=filtDWellPPScenarios[filtDWellPPScenarios['ID']==ID]['Pressures']
  maxPressureList.append
dWellPPScenarios['MaxPressure']=dWellPPScenarios['ID'].map(maxPressureDict)
dWellPPScenarios=dWellPPScenarios.sort_values(by='MaxPressure',ascending=False)

In [0]:
# Select wells that only have greater than a 0.1 psi stress increase
deepWindowedWells=allDeep[allDeep['Stresses']>0.1]['ID'].unique()
allDeepWindowed=allDeep[allDeep['ID'].isin(deepWindowedWells)]
# Get maximum pressure or stress for each well, sort by that
maxList=[]
for ID in deepWindowedWells:
  # get maximum row
  maxRow=allDeepWindowed[allDeepWindowed['ID']==ID]['Stresses'].idxmax()
  maxList.append([allDeepWindowed.iloc[maxRow]['ID'],allDeepWindowed.iloc[maxRow]['Stresses']])
wellOrdered=pd.DataFrame(maxList,columns=['ID','Stresses']).sort_values(by='Stresses')
IDOrder=wellOrdered['ID'].to_list()
sortedDeepWindowed=allDeepWindowed.sort_values(by="ID", key=lambda column: column.map(lambda e: IDOrder.index(e)), inplace=True)

In [0]:
fig,axes=plt.subplots(1,1, sharex=True, figsize=(30,200))
# Relative contributions
sns.stripplot(data=allDeep,x='Stresses',y='WellName',hue='Physics',dodge=True,jitter=True,alpha=0.1,ax=axes)
axes.set_title('Deep Well Stresses')
axes.set_xlabel('Stress (PSI)')

In [0]:
# Concatenate pore pressure and poroelastic results
sWellPPScenarios['Physics']='Pore Pressure'
sWellPEScenarios['Physics']='Poroelastic'
sWellPPScenarios['Stresses']=sWellPPScenarios['Pressures']
allShallow=pd.concat(objs=[sWellPPScenarios,sWellPEScenarios[sWellPEScenarios['ID'].isin(sWellPPScenarios['ID'])]])
allShallow=allShallow.merge(shallowPEWells,on='ID')


In [0]:
fig,axes=plt.subplots(1,1, sharex=True, figsize=(30,10))
# Relative contributions
sns.stripplot(data=allShallow,x='Stresses',y='WellName',hue='Physics',dodge=True,jitter=True,alpha=0.1,ax=axes)
axes.set_title('Shallow Well Stresses')
axes.set_xlabel('Stress (PSI)')

In [0]:
# Plot results
fig,axes=plt.subplots(3,1, sharex=True, figsize=(60,20))
gp.disaggregationPlot(axes[0],dWellPPScenarios,dWellPEScenarios,deepPEWells,'Deep')
gp.disaggregationPlot(axes[1],sWellPPScenarios,sWellPEScenarios,shallowPEWells,'Shallow')
# Concatenate pore pressure and poroelastic results


In [0]:
# 

dayDVec=np.linspace(deepM.injOT, deepM.injOT+deepM.injDT*(deepM.injNT-1.),deepM.injNT)
dPPTimeSeriesSum=np.sum(dPPTimeSeries,axis=1)
realization=np.linspace(0,499,500)
nwC=deepPPWells.shape[0]
dTPPDF=pd.DataFrame(columns=['Days','Realization','TotalPressure'])
dPPDF=pd.DataFrame(columns=['Days','Realization','ID','Pressure','TotalPressure'])
dTPPDF['Days']=np.tile(dayDVec,deepM.nReal)
dTPPDF['TotalPressure']=np.reshape(dPPTimeSeriesSum[:,:],deepM.injNT*deepM.nReal,order='F')
dTPPDF['Realization']=np.tile(realization,deepM.injNT)
dTPPDF=gi.getDates(dTPPDF,deepM.epoch)

In [0]:
fig,axes=plt.subplots(2,1, sharex=True, figsize=(40,15))

sns.lineplot(ax=axes[0],x='Date',y='TotalPressure',hue='Realization',data=dTPPDF)
sns.scatterplot(data=filtEQs,ax=axes[0],x='Date',y='Magnitude',s=300,marker='*')
axes[0].set_title('Deep Pressure Histories')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Pressure (PSI)')
sns.lineplot(ax=axes[1],x='Date',y='TotalPressure',hue='Realization',data=sTPPDF)
sns.scatterplot(data=filtEQs,ax=axes[1],x='Date',y='Magnitude',s=300,marker='*')
plt.show()